In [ ]:
import PyPDF2
import re
from IPython.display import display, Markdown

import os
import openai

import json
credentials = json.load(open('keys.json'))
openai.organization = credentials['organization_key']
openai.api_key = credentials['api_key']

In [ ]:
class Section:
    def __init__(self, number, title, text, lo, gpt_notes = ""):
        self.number = number
        self.title = title
        self.text = text
        self.lo = lo
        self.gpt_notes = gpt_notes

    def __str__(self):
        lo_string = "\n\n".join(f'*{lo}*' for lo in self.lo)
        return f'**{self.number} {self.title}**\n\n{lo_string}\n\n{self.gpt_notes}'
    
    def display(self):
        display(Markdown(str(self)))

In [ ]:
def get_pdf_content(file_path):
    pdf_file_obj = open(file_path, 'rb')
    reader = PyPDF2.PdfReader(pdf_file_obj)

    content = ""
    for page_num in range(len(reader.pages)):
        page_obj = reader.pages[page_num]
        content += page_obj.extract_text()

    pdf_file_obj.close()

    return content

def find_sections(content, section_num):
    pattern = fr"({section_num}\.\d+(\.\d+)*)(.*?)(?=({section_num}\.\d+(\.\d+)*)|$)"
    sections_raw = re.findall(pattern, content, re.DOTALL)
    sections = []
    for t in sections_raw:
        number = t[0]
        title = t[2].split("\n")[0].lstrip().rstrip()
        text = t[2]
        text = text.replace(title, '').replace("This element addresses the following learning objective of this course:\n", '').replace("This element addresses the following learning objectives of this course:", '').replace("\n", " ").replace("● ", "").replace("  ", " ").lstrip().rstrip()
        text_without_lo = remove_lo(text)
        lo = get_lo(text)
        sections.append(Section(number, title, text_without_lo, lo))
    return sections

def get_lo(content):
    pattern = r"(LO\d+:.*?[.!?])"
    los = re.findall(pattern, content)
    return [re.sub(r'LO\d+: ', '', lo) for lo in los]

def remove_lo(content):
    pattern = r"(LO\d+:.*?[.!?])"
    return re.sub(pattern, '', content, flags=re.MULTILINE)

In [ ]:
# Read in the PDF
# UPDATE FILE AND SECTION NUMBER HERE
pdf_content = get_pdf_content('Week 10 transcript (updated June 2021).pdf')
section_num = "10"

In [ ]:
# Overwrite all the workd that is done. DO NOT DO THIS UNLESS YOU WANT TO START OVER
sections = find_sections(pdf_content, section_num)

In [ ]:
for section in sections:
    if section.text == "" or "on the following prompt" in section.text:
        continue
    if section.gpt_notes != "":
        continue
    else:
        gpt_content = f"""Topic: {section.title}
        Detail Level: I would like something between high-level summary and detailed notes.
        Learning Objectives: {'; '.join(section.lo)}
        Format: I prefer the notes to be in bullet-point or table format.
        Course Transcript: {section.text}
        Purpose: The notes are for my personal understanding and to facilitate in class discussion with my peers."""
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", 
                 "content": gpt_content}
                 ])
        section.gpt_notes = completion.choices[0].message.content

    print(section.title, "- Complete")

Biases in Decision Making, Part 1 - Complete
 - Complete
 - Complete
Common Biases Overview - Complete
Self-Serving Bias (SSB) - Complete
How Self-Serving Bias (SSB) Surfaces - Complete
Ambiguity Effect - Complete
Endowment Effect - Complete
Endowment Effect vs. Reality - Complete
Hindsight Bias - Complete
Hindsight Bias Consequences● Hindsight bias obviously can have pernicious consequences for leadership and - Complete
Bias Interview - Complete
Make Choices Consistent With - Complete
Dead Conventions - Complete
Similarities and DifferencesThe 1950s advertisements Steve references can be found below the video. - Complete
Fallacies in Decision Making - Complete
Demonstrating Fallacies - Complete
 - Complete
 - Complete
 - Complete
 - Complete
The Powerful Anecdote - Complete
The Appeal to Novelty - Complete
The Slippery Slope - Complete
Bad Decisions Are Human: - Complete
Bad Decisions Are Human - Complete
The Rules of Inference - Complete
Applied Examples of the Rules of Inference - C

In [7]:
# Copy and past from the output of the cell to your notes

for section in sections:
    if section.gpt_notes == "":
        # print(section.title)
        display(Markdown('**' + str(section.title) + '**'))
    else:
        section.display()

**10.1 Biases in Decision Making, Part 1**



- Biases in decision making are inherent cognitive shortcuts or mental patterns that can influence our judgments and decisions.
- Confirmation bias: tendency to seek and interpret information in a way that confirms our preexisting beliefs or hypotheses.
- Dunning-Kruger effect: cognitive bias where individuals with low ability at a particular task overestimate their ability, while those with high ability may underestimate their own competence.
- Cognitive dissonance: psychological discomfort caused by holding conflicting beliefs or values, leading to a desire to reduce the discomfort through behavior or attitude changes.
- Philosophy of science: provides a framework for understanding scientific methods, theories, and the nature of scientific inquiry.
- Loss aversion: the tendency to strongly prefer avoiding losses compared to acquiring equivalent gains, often leading to irrational decision making.
- Overconfidence: the tendency to have excessive confidence in one's own abilities or judgments, often leading to errors in decision making.
- These biases can shape our worldview, affect our decision making processes, and impact the outcomes of our choices.
- Awareness of biases is important to mitigate their negative impact and make more informed decisions.
- The videos provide insights into the concepts, consequences, and real-life examples related to these biases.

**10.2 **



- Biases in decision making can be both fascinating and funny to study
- There is an ongoing debate about whether certain behaviors are rational or irrational
- Example: People buying a car after test driving only two or three cars
- The idea of rationality can be seen as "just so stories" and can be misleading
- The focus should be on understanding and identifying common biases in decision making
- The goal is to use data to address biases for the benefit of the decision maker
- Example: Two identical cancer patients making a decision about an experimental treatment
- The emphasis is not on who made the right or rational choice, but on using data to save lives
- Data alone may not overcome biases, but it can make a difference when presented correctly at the right moment in the decision process
- There will be experiments in the future to identify and utilize valuable windows of opportunity in decision-making processes for both individuals and businesses.

**10.3 **



- Biases in decision making
- Confirmation bias: seeking evidence that supports our hypothesis and ignoring evidence that conflicts with it
- Examples: personal health hypothesis, belief in a company's stock, cherry picking data in scientific papers
- Confirmation bias is not intentional, but a cognitive and emotional mistake
- Splitting and bolstering: making alternatives seem more different than they are and bolstering our chosen option
- Examples: choosing between two cars, corporate mergers and acquisitions
- Reasons for splitting and bolstering: reducing cognitive dissonance, reducing discomfort and uncertainty, reducing appearance of ambivalence
- Splitting and bolstering can be a management tactic to appear decisive and provide certainty
- Examples: leadership in uncertain situations, political decision making

**Common Biases Overview**

**10.4.1 Common Biases Overview**

*Navigate organizational, personal, legal, and ethical constraints to facilitate better decision making and improve communication.*

Common Biases Overview:

1. Confirmation Bias:
- The tendency to search for, interpret, and remember information in a way that confirms one's preexisting beliefs.
- People tend to ignore or dismiss information that challenges their existing views.
- Example: A person who believes that climate change is a hoax may only seek out information that supports their belief and ignore research and evidence to the contrary.

2. Anchoring Bias:
- The tendency to rely too heavily on the first piece of information encountered when making a decision.
- This information acts as an "anchor" and influences subsequent judgments.
- Example: Pricing an item higher than its actual value to make a later price seem like a better deal.

3. Availability Bias:
- The tendency to overestimate the likelihood of events based on their availability in memory.
- People rely on immediate examples that come to mind rather than considering complete data or statistical probabilities.
- Example: Someone might assume that plane crashes are more frequent than car accidents because news headlines about plane crashes tend to be more attention-grabbing.

4. Confirmation Bias:
- The tendency to seek out and interpret information that confirms one's existing beliefs while ignoring contradictory evidence.
- This bias can lead to a reinforcement of preexisting beliefs and hinder openness to alternative perspectives or information.
- Example: A person who strongly believes in a particular political ideology may only consume media that aligns with their beliefs, thus reinforcing their views.

5. Overconfidence Bias:
- The tendency to overestimate one's abilities, knowledge, or wisdom.
- People often have an unwarranted confidence in their judgments or predictions.
- Example: A student may perceive themselves as better prepared for an exam than they actually are, leading to poor performance.

6. Confirmation Bias:
- The tendency to interpret or remember information in a way that confirms our existing beliefs or theories.
- This can lead to selective perception and memory distortion.
- Example: A person who values punctuality may perceive others as being late more often than they actually are.

7. Halo Effect:
- The tendency to generalize a positive impression of someone or something based on one specific characteristic or trait.
- This can lead to biased judgments and assumptions about unrelated attributes.
- Example: Assuming that a physically attractive person must also be intelligent and kind.

8. Hindsight Bias:
- The tendency to see past events as more predictable than they actually were.
- This bias can lead to an overestimation of one's ability to predict future events and an underestimation of luck or chance.
- Example: After a stock market crash, people may claim they "knew it was going to happen" when in reality they were not able to predict it accurately.

Note: This is not an exhaustive list of biases, and there are many other biases that can influence decision-making and communication.

...

**10.15.1 Applied Examples of the Rules of Inference**

*Justify an analytic approach that informs decision making.*

*Navigate organizational, personal, legal, and ethical constraints to facilitate better decision making and improve communication.*

- John Stuart Mill's four rules of inference:
   1. Method of agreement: If x is a necessary cause of y, you must observe x every time you observe y.
   2. Method of difference: If x is a sufficient cause of y, then when x is present, y will be observed, and when x is absent, y will not be observed.
   3. Joint method of agreement and difference: If x is a necessary and sufficient cause of y, it must be present when y is observed and absent when y is not observed.
   4. Method of concomitant variation: If x and y vary together in a consistent manner, there is likely a causal relationship between them.

- Applied example of the method of agreement:
   - Claim: A PhD is a necessary cause of being a professor.
   - In order to observe someone as a professor (y), they must have a PhD (x).
   - Having a PhD is necessary to be a professor, but not sufficient.
   - Not all individuals with a PhD are professors.

- Example of the importance of the temporal relationship:
   - Claim: Investor attitudes influence CEO behavior.
   - Investor attitudes (x) exist prior to CEO behavior (y), satisfying the requirement that x comes before y.
   - However, there may be a different story where CEO behavior influences investor opinion, reversing the causal relationship.
   - The relationship between x and y can be more complex than initially posited, highlighting the need to be clear about the assumptions underlying the causal story.

- Conclusion:
   - Articulating a causal story requires clarity about the assumptions that form the foundation of the story.
   - Different causal relationships may exist, challenging initial assumptions.
   - Understanding and applying the rules of inference can facilitate better decision making by considering various factors and constraints.
   - Improved communication can be achieved by justifying an analytic approach based on these rules.

**10.15.2 John Stuart Mill's Rules of Inference**

*Justify an analytic approach that informs decision making.*

*Navigateorganizational, personal, legal, and ethical constraints to facilitate better decision making and improve communication.*

- John Stuart Mill's Rules of Inference are methods for identifying causes and effects in a phenomenon under investigation.
- The Method of Difference:
   - Compares two instances where the phenomenon occurs and does not occur, with all circumstances except one being common.
   - The differing circumstance is considered the cause or a necessary part of the cause of the phenomenon.
   - It doesn't exclude the possibility that the differing circumstance is an effect of the phenomenon.
- The Joint Method of Agreement and Difference:
   - Compares multiple instances where the phenomenon occurs and multiple instances where it does not occur.
   - The only common circumstance in the instances where it occurs is considered the cause or a necessary part of the cause of the phenomenon.
   - The differing circumstance in the instances where it does not occur is considered the cause or a necessary part of the cause of the phenomenon.
- The Method of Concomitant Variation:
   - Focuses on the quantitative variation of two connected phenomena.
   - When variations in one phenomenon are consistently related to variations in another, a causal connection is inferred.
   - The broader the set of circumstances in which this relationship is observed, the stronger the inference.
- These rules help in justifying an analytic approach for decision-making and navigating constraints to improve communication.

**10.16 **



- Pragmatic limits to inferential rules
  - Two big limits on inference in a business setting
- First limit: John Stuart Mill (JSM) presupposes a good candidate list of causes
  - The list of causes should include the important cause that drives the phenomenon
  - Where does the list of candidates come from?
    - Informed guess, previous work, history, opinion
  - If the desired cause is missing from the list, there is a limit to inference
- Second limit: JSM presupposes relatively simple causal relationships
  - JSM's rules don't handle complex conjunctural causation well
  - Complex conjunctural causation: many factors causing an outcome together, possibly on different time frames
  - Example: oncogene is the tumor initiator, but there are other factors like high fat diet and low exercise lifestyle as tumor promoters
  - JSM's method may not be able to identify complex causal relationships
- Managing complex causation is possible with advanced statistics, but not covered in this course
- Purpose: Understand the challenges of isolating and explaining causation findings

**10.17 Expert Mind vs. Beginner Mind**



- The concept of expert mind vs. beginner mind is discussed in relation to pattern matching and interpretation of data.
- The expert mind is described as being highly skilled at pattern matching and quickly identifying patterns in data.
- The beginner mind, on the other hand, lacks established patterns and is more open to exploring and discovering patterns.
- The value of the beginner mind is emphasized, particularly in dealing with pragmatic paradoxes or optical illusions.
- The idea of adopting a beginner's mind or 11-month-old baby attitude is encouraged as an exercise to explore alternate perspectives.
- The purpose of the discussion is to suggest a simpler and more pragmatic way to understand the interpretation of patterns in the mind.

**10.18 Premortum Analysis**

*Navigate organizational, personal, legal, and ethical constraints to facilitate better decision making and improve communication .*

- Premortem analysis is a technique introduced by Nobel Laureate Daniel Kahneman to eliminate thinking biases and improve decision making.
- It involves conducting a hypothetical postmortem of a decision or project before it actually takes place.
- The purpose of the premortem analysis is to identify potential problems and failures that might occur in the future and develop strategies to prevent or mitigate them.
- The technique helps in considering different perspectives and viewpoints, which may reveal previously overlooked risks or challenges.
- Premortem analysis allows for open and honest communication among team members, encouraging everyone to speak up and share their concerns or doubts.
- This technique helps to counteract the optimistic bias and overconfidence that often lead to flawed decision making.
- By encouraging individuals to actively criticize and question their own assumptions and plans, premortem analysis facilitates a more thorough evaluation of potential risks and improves decision making.
- Premortem analysis should be conducted in a safe and constructive environment, where participants are encouraged to provide feedback without fear of judgment or repercussions.
- The technique can be applied to various contexts, such as business projects, personal decisions, or policy making.
- Ethical considerations should be taken into account during the premortem analysis to ensure that the process respects the rights and well-being of individuals involved.
- Legal constraints should also be considered, especially when conducting premortem analysis in organizations or sensitive contexts.
- In summary, premortem analysis is a powerful technique that helps overcome biases, improve decision making, and enhance communication by identifying and addressing potential problems and failures before they occur.